<a href="https://colab.research.google.com/github/SSTalha/PhiTuning/blob/main/FashionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqqq pip
!pip install bitsandbytes==0.40.0.post4
!pip install -qqq torch==2.1.0
!pip install -qqq -U transformers==4.30.0
!pip install -qqq -U peft==0.6.0
!pip install -qqq -U accelerate==0.24.1
!pip install -qqq datasets==2.14.6
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00


In [ ]:
import torch
import transformers
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import bitsandbytes as bnb
from peft import(prepare_model_for_kbit_training,
                 PeftModel,
                 LoraConfig,
                 get_peft_model)
from transformers import (TrainingArguments,
                          Trainer,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          AutoModelForCausalLM,
                          DataCollatorForLanguageModeling)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda122.so


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 122
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda122.so...


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Load Model

In [ ]:
MODEL = "typeof/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(MODEL , trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Quantization

In [ ]:
bnb_config = BitsAndBytesConfig(
    Load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map = {"":0},
    trust_remote_code=True,
    quantization_config = bnb_config
)
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

configuration_mixformer_sequential.py:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/typeof/phi-1_5:
- configuration_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_mixformer_sequential.py:   0%|          | 0.00/32.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/typeof/phi-1_5:
- modeling_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [ ]:
model

MixFormerSequentialForCausalLM(
  (layers): Sequential(
    (0): Embedding(
      (wte): Embedding(51200, 2048)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (1): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (resid_dropout): Dropout(p=0.0, inplace=False)
      (mixer): MHA(
        (rotary_emb): RotaryEmbedding()
        (Wqkv): Linear(in_features=2048, out_features=6144, bias=True)
        (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (inner_attn): SelfAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
        (inner_cross_attn): CrossAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (mlp): MLP(
        (fc1): Linear(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        (act): NewGELUActivation()
      )
    )
    (2): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwis

LoRA (Low Rank Adaptation) Config

In [ ]:
from bitsandbytes.nn.modules import Linear8bitLt, Linear4bit
import torch.nn.init as init
from contextlib import contextmanager

def noop (x=None, *args, **kwargs ):
  "Do nothing"
  return x

@contextmanager
def no_kaiming():
  old_iku = init.kaiming_uniform_
  init.kaiming_uniform_ = noop
  try : yield
  finally : init.kaiming_uniform_ = old_iku


_old_8init = Linear8bitLt.__init__
_old_4init = Linear4bit.__init__

def _new_4init(self, input_features, output_features, bias = True,
               device = None, **kwargs):
  with no_kaiming():
    return _old_4init(self, input_features, output_features, bias = bias,
                      device = device, **kwargs)


def _new_8init(self, input_features, output_features, bias = True, has_fp16_weights = True,
            memory_efficient_backward = False, threshold = 0.0, index = None, device = None):
  with no_kaiming():
    return _new_8init(self, input_features, output_features, bias = bias, has_fp16_weights = has_fp16_weights,
            memory_efficient_backward = memory_efficient_backward, threshold = threshold, index = index, device = device)


Linear8bitLt.__init__ = _new_8init
Linear4bit.__init__ = _new_4init

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["wqkv", "out_proj","fc1","fc2"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model , lora_config)

model.print_trainable_parameters

<bound method PeftModel.print_trainable_parameters of PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): MixFormerSequentialForCausalLM(
          (layers): Sequential(
            (0): Embedding(
              (wte): Embedding(51200, 2048)
              (drop): Dropout(p=0.0, inplace=False)
            )
            (1): ParallelBlock(
              (ln): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
              (resid_dropout): Dropout(p=0.0, inplace=False)
              (mixer): MHA(
                (rotary_emb): RotaryEmbedding()
                (Wqkv): Linear(in_features=2048, out_features=6144, bias=True)
                (out_proj): Linear(
                  in_features=2048, out_features=2048, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    

# Load Dataset

In [ ]:
data = load_dataset("csv", data_files = "FashionDataset.csv")

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['ID', 'Gender', 'Skin Tone', 'Query', 'Recommendation'],
        num_rows: 430
    })
})

In [ ]:
data["train"][0]

{'ID': 1,
 'Gender': 'Male',
 'Skin Tone': 'Black',
 'Query': 'Suggest a Formal attire for a wedding',
 'Recommendation': 'Black tailored suit with a white dress shirt and a silver tie'}

Tokenize Dataset

In [ ]:
def generate_recommendation(data_point):
  return f"""
  <human>: {data_point["Query"]}, {data_point["Skin Tone"]}, {data_point["Gender"]}
  <assistant>: {data_point["Recommendation"]}
  """.strip()

def generate_and_tokenize_recommendation(data_point):
  recommendation = generate_recommendation(data_point)
  tokenized_full_prompt = tokenizer(recommendation, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_recommendation)

Map:   0%|          | 0/430 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['ID', 'Gender', 'Skin Tone', 'Query', 'Recommendation', 'input_ids', 'attention_mask'],
    num_rows: 430
})

# FineTune the Model

In [ ]:
training_args = TrainingArguments(
    output_dir = "phi-1_5-Fashionchat-finetuned",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps=4,
    num_train_epochs=8,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=50,
    lr_scheduler_type="cosine",
    save_strategy = "epoch",
    warmup_ratio=0.05,
    optim="paged_adamw_8bit",
)


In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

Step,Training Loss
50,2.307600
100,0.894800
150,0.692300
200,0.545000
250,0.451100
300,0.345900
350,0.276500
400,0.225200


TrainOutput(global_step=424, training_loss=0.6886319911704873, metrics={'train_runtime': 307.4626, 'train_samples_per_second': 11.188, 'train_steps_per_second': 1.379, 'total_flos': 1037522076229632.0, 'train_loss': 0.6886319911704873, 'epoch': 7.89})

# Run FineTuned Model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: Suggest a party casual wear for tan male
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      # attention_mask = encoding.attention_mask,
      generation_config = generation_config,
      top_k=50,
      temperature=0.8,
  )

first_recommendation = tokenizer.decode(outputs[0], skip_special_tokens=True).split("<assistant>:")[1].strip()

first_full_stop_index = first_recommendation.find(".")

# Limit the response to the content before the first full stop
formatted_response = f"{prompt} {first_recommendation[:first_full_stop_index + 1]}" if first_full_stop_index != -1 else f"{prompt} {first_recommendation}"

print(formatted_response)



<human>: Suggest a party casual wear for tan male
<assistant>: Light khaki shorts with a light blue button-up shirt and tan flip flops.
CPU times: user 12.3 s, sys: 39.5 ms, total: 12.4 s
Wall time: 18.4 s
